In [3]:
# 패션 MNIST 데이터 가져오기
import keras
(train_input, train_target), (test_input, test_target) =\
     keras.datasets.fashion_mnist.load_data()

In [4]:
# Conv2D에의 데이터는 반드시 채널 차원이 필요
train_input = train_input.reshape(-1, 28, 28, 1)

In [5]:
# 정규화 - 0~1
train_scaled = train_input / 255.0

In [6]:
# 학습세트와 검증세트 분리
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 층 구성 
model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28,1)))